# 03 – Dispatcher & Query


Maps `QuerySpec` → concrete API calls and applies deterministic filters in C#.


In [ ]:
using semantic_kernel_app;
using Microsoft.Extensions.Logging;

public class QueryDispatcher : IQueryDispatcher
{
    private readonly IEmployeeApi _api;
    private readonly ILogger<QueryDispatcher> _log;

    public QueryDispatcher(IEmployeeApi api, ILogger<QueryDispatcher> log)
    {
        _api = api; _log = log;
    }

    public async Task<object> ExecuteAsync(QuerySpec spec, CancellationToken ct = default)
    {
        var employees = await _api.GetEmployeesAsync(ct);

        IEnumerable<Employee> q = employees;

        if (spec.Slots.Names?.Length > 0)
        {
            var set = spec.Slots.Names.Select(n=>n.ToLowerInvariant()).ToHashSet();
            q = q.Where(e => set.Contains(e.DisplayName.Split(' ')[0].ToLowerInvariant()));
        }
        if (!string.IsNullOrWhiteSpace(spec.Slots.Department))
        {
            q = q.Where(e => e.Department.Equals(spec.Slots.Department, StringComparison.OrdinalIgnoreCase));
        }
        if (!string.IsNullOrWhiteSpace(spec.Slots.Role))
        {
            // naive contains match
            q = q.Where(e => e.Role.Contains(spec.Slots.Role!, StringComparison.OrdinalIgnoreCase));
        }
        if (spec.Intent == Intent.FilterByHireDate)
        {
            var op = spec.Slots.Operator;
            if (op == "before" && spec.Slots.Date is DateTime d1) q = q.Where(e => e.OriginalHireDate < d1);
            if (op == "after" && spec.Slots.Date is DateTime d2) q = q.Where(e => e.OriginalHireDate > d2);
            if (op == "between" && spec.Slots.Range is (DateTime s, DateTime e))
                q = q.Where(x => x.OriginalHireDate >= s && x.OriginalHireDate <= e);
        }

        return q.ToList();
    }
}

// demo
var disp = new QueryDispatcher(new InMemoryEmployeeApi(new Microsoft.Extensions.Caching.Memory.MemoryCache(new Microsoft.Extensions.Caching.Memory.MemoryCacheOptions()), new AppConfig()),
    LoggerFactory.Create(b=>b.AddConsole()).CreateLogger<QueryDispatcher>());

var spec = new QuerySpec(Intent.FilterByHireDate,
    new Slots(Department:"engineering", Operator:"after", Date:new DateTime(2020,1,1)));

var result = await disp.ExecuteAsync(spec);
Console.WriteLine(System.Text.Json.JsonSerializer.Serialize(result));
